In [2]:
let text = r#"
Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
"#;
let lines: Vec<&str> = text.trim().lines().collect();
lines

["Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53", "Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19", "Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1", "Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83", "Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36", "Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"]

In [3]:
#[derive(Debug)]
struct Card {
    pub id: i32,
    winning_numbers: Vec<i32>,
    guess_numbers: Vec<i32>,
}

impl Card {
    fn from_string(s: &str) -> Card {
        let parts: Vec<&str> = s.splitn(2, ':').collect();
        let card = parts[0];
        let numbers = parts[1];

        let id: i32 = card
            .split_whitespace()
            .nth(1)
            .expect("Error parsing Card ID")
            .parse()
            .expect("Error converting Card ID to i32");
        
        let number_parts: Vec<&str> = numbers.splitn(2, '|').collect();
        let winning_numbers: Vec<i32> = number_parts[0]
            .split_whitespace()
            .map(|n| n.parse().expect("Error converting winning number to i32"))
            .collect();
        let guess_numbers: Vec<i32> = number_parts[1]
            .split_whitespace()
            .map(|n| n.parse().expect("Error converting guess number to i32"))
            .collect();
        Card {
            id,
            winning_numbers,
            guess_numbers
        }
    }

    fn copies(&self) -> Vec<i32> {
        let copy_range = self.guess_numbers
            .iter()
            .filter(|n| self.winning_numbers.contains(n))
            .count();
        let start = self.id + 1;
        let stop = start + copy_range as i32;
        (start..stop).collect()
    }
}

let c = Card::from_string(&lines[0]);
c

Card { id: 1, winning_numbers: [41, 48, 83, 86, 17], guess_numbers: [83, 86, 6, 31, 17, 9, 48, 53] }

In [4]:
c.copies()

[2, 3, 4, 5]

In [5]:
use std::collections::HashMap;

let mut cards: Vec<Card> = Vec::new();
for line in &lines {
    cards.push(Card::from_string(line));
}

// basically {card.id: 1 for card in cards}
let mut counts: HashMap<i32, i32> = cards.iter().map(|card| (card.id, 1)).collect();

for card in &cards {
    let mult = *counts.get(&card.id).unwrap();
    for copy in card.copies() {
        if let Some(count) = counts.get_mut(&copy) {
            *count += mult;
        }
    }
}

counts

{5: 14, 6: 1, 1: 1, 2: 2, 3: 4, 4: 8}

In [6]:
counts.values().into_iter().sum::<i32>()

30